In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import bintrees

import entities
import solutions
import checker

# Пересечение множества отрезков
Перед нами стоит задача: дано множество $S$, состоящее из $n$ отрезков на плоскости. Нам нужно найти все точки их пересечений. 

Очевидно, что наивный алгоритм, перебирающий все пары отрезков, имеет сложность $O(n^2)$. В некотором смысле это оптимальная оценка, так как в случае, когда каждая пара отрезков из множества пересекается, любой алгоритм займет $\Omega(n^2)$ времени. Однако на практике такие ситуации встречаются редко, обычно пересечений гораздо меньше. Поэтому хотелось бы уметь решать данную задачу быстрее в таких ситуациях. Другими словами, нужен алгоритм, время работы которого будет зависеть не только от $n$, но и от количества пересечений. Такие алгоритмы называются *output-sensitive*. 

В этом конспекте мы рассмотрим алгоритм Бентли-Оттмана (*англ. Bentley-Ottmann*), который позволяет решать поставленную задачу за время $O((n + I)\log{n})$, где $I$ - количество персечений.

Перед тем, как приступить к изучению этого материала, настоятельно рекомендуется ознакомиться с конспектом про [афинное пространство](https://github.com/CT-18/cg/tree/5_affine_space), а именно с предикатом "левый поворот" и задачей о пересечении двух отрезков.

## Алгоритм Бентли-Оттмана
<img src="images/sweep_line.png" style="float: right"/>

Посмотрим на множество $S$. Нам нужно найти такое свойство или правило, по которому мы смогли бы отметать отрезки, которые точно не имеют общих точек, и не проверять их на пересечение.

Начнем с достаточно простого наблюдения. Спроецируем все отрезки на ось $X$. Заметим, что отрезки, проекции которых не пересекаются, не могут иметь общих точек. А для отрезков, проекции которых накладываются друг на друга, существует параллельная оси $Y$ прямая, пересекающая их. 

Представим прямую $l$, скользящую по плоскости параллельно оси $Y$, начинающую свое движение из $-\infty$. Будем называть её заметающей прямой (*sweep line*). Тогда, принимая во внимание наше наблюдение, мы можем утверждать, что пересекаются только такие отрезки, для которых существует положение прямой $l$, при котором она одновременно их пересекает. Таким образом, во время движения $l$ вдоль плоскости мы сможем отслеживать все отрезки, пересекающие её в один момент времени, и находить их пересечения.

Будем называть статусом (*status*) заметающей прямой множество отрезков, пересекающих $l$ в данный момент времени, и событиями (*event points*) - точки, в которых статус меняется. Очевидно, что в нашем случае это точки начал и концов отрезков. 

Заметим, что алгоритм совершает какие-то дейтвия только при достижении $l$ события: обновляется статус прямой и происходят проверки отрезков на наличие пересечений. Например, если событие - "левый конец отрезка" $s_i$, то это значит, что $s_i$ начал пересекать заметающую прямую и должен быть добавен в статус. После добавления $s_i$ проверяется на пересечение с отрезками, уже имеющимися в статусе. При достижении события "правый конец", отрезок перестает пересекать $l$ и должен быть удален из статуса. Таким образом, мы проверяем на пересечение только такие отрезки, для которых существует вертикальная прямая, пересекающая их.

Однако такой алгоритм все еще не output-sensitive, поскольку есть ситуации, при которых происходит проверка $O(n^2)$ пар отрезков, хотя пересечений на самом деле много меньше: простейший пример - множество горизонтальных отрезков, пересекающих ось $Y$. Проблема в том, что два отрезка, пересекающих заметающую прямую, могут находиться далеко друг от друга в вертикальном направлении. 

<img src="images/intersection_change_status.png" style="float: right"/>
Упорядочим отрезки, пересекающие $l$, снизу вверх: это поможет нам понять, какие отрезки находятся близко. Теперь будем проверять на пересечение только ближайших соседей, то есть каждый новодобавленный в статус отрезок будем проверять только с его непосредственными соседями снизу и сверху, если они есть. Позже у этого отрезка могут измениться соседи, и тогда они снова будут проверены на пересечение. Такой подход отразится на статусе заметающей прямой: теперь статус - это упорядоченная последовательность отрезков, пересекающих $l$. Соответственно, теперь он меняется не только при достижении концов отрезков, но и в точках пересечения, так как в них меняется порядок пересекающихся отрезков. Следовательно, мы должны проверить эти два отрезка на пересечение с их новыми соседями. Таким образом, теперь у нас появился третий тип событий - точки пересечения отрезков.

Итак, мы разработали так называемый *plane sweep algorithm*. Для полного понимания принципа его работы, ниже представлена визуализация движения заметающей прямой:
<img src="images/sweep_line_visualization.gif" style="float: center"/>

Теперь давайте подробнее разберем, что именно должно происходить при достижении заметающей прямой каждого типа события. Так как статус, как ранее было сказано, это упорядоченный набор пересекающих заметающую прямую отрезков в конкретный момент времени, то два отрезка являются соседними в статусе, если они являются ближайшими вдоль $l$.
1. <img src="images/upper_endpoint.png" style="float: right"/> Если событие - начало отрезка, то это значит, что новый отрезок начал пересекать заметающую прямую. Поэтому он должен быть добавлен в статус и проверен на пересечение с двумя своими непосредственными соседями сверху и снизу, если они есть. Нас интересуют только пересечения правее заметающей прямой, так как если пересение находится левее, то оно уже было обнаружено (этот достаточно очевидный факт будет доказан чуть позже). Например, если отрезки $s_k$ и $s_l$ были соседними в статусе, и в какой-то момент между ними появился новый отрезок $s_i$, то мы должны проверить, не пересекается ли $s_i$ с $s_k$ и $s_i$ с $s_l$. Если мы находим точку пересечения левее $l$, то мы должны запомнить это событие (позже мы разберем подробнее, что это значит) и впоследствии обработать его. <br><br>
2. <img src="images/intersection_point.png" style="float: right"/> Если событие - точка пересечения, то это значит, что эти отрезки поменяли свой порядок вдоль $l$. У каждого из них появился как максимум один новый сосед, с которым нужно проверить соответствующий отрезок на пересечение. Опять же нас интересуют только пересечения правее заметающей прямой (по той же причине). Приведём пример: предположим, что четыре отрезка идут в статусе в порядке $s_i$, $s_k$, $s_j$, $s_m$ в момент, когда $l$ достигает точки пересечения $s_k$ и $s_j$ (как на рисунке справа). В следующий момент эти два отрезка меняются в статусе местами, и мы должны проверить на перечечение $s_i$ с $s_j$ и $s_k$ с $s_m$. Если мы находим новые точки пересечений, то запоминаем их для дальнейшей обрабртки. Однако здесь стоит не забывать о том, что эти новые пересечения мы могли уже обнаружить ранее. <br><br>
3. <img src="images/lower_endpoint.png" style="float: right"/> Если событие - конец отрезка, то это значит, что он перестал пересекать заметающую прямую. Следовательно, его соседи сверху и снизу, если они оба есть, теперь являются ближайшими вдоль $l$ и должны быть проверены на пересечение. Если мы обнаружили пересечение правее заметающей прямой, то, опять же, запоминаем это событие для дальнейшей обработки. Предположим, что в статусе отрезки $s_k$, $s_j$ и $s_m$ идут в указанном порядке, когда заметающая прямая достигает канца отрезка $s_j$. Тогда отрезки $s_k$ и $s_m$ должны быть проверены на пересечение. Здесь снова стоит помнить, что новое событие "пересечение отрезков" могло быть уже обраружено ранее.

Теперь давайте обсудим, какие структуры данных нам понадобятся для реализации алгоритма. На самом деле их всего две: очередь событий $\Omega$ и статус $T$. 

Как раньше было показано, алгоритм совершает какие-то действия только при достижении заметающей прямой какого-либо события. Следовательно, программно сэмитировать движение $l$ мы можем просто рассматривая события в том порядке, в котором прямая настигла бы их по ходу своего движения справа налево. Определим отношение "меньше" для двух событий $p$ и $q$: будем считать, что событие "левый конец" меньше события "пересечение", а последнее меньше собятия "правый конец". Если события одного типа, то 
$$ p < q \Longleftrightarrow \left[ 
  \begin{align} 
    &p_x < q_x \\ 
    &p_x = q_x \wedge p_y < q_y \\ 
  \end{align} 
\right. $$
Таким образом, для очереди нам нужна такая структура данных, в которой мы могли бы хранить события в соответствии с определённым выше порядком. Кроме того надо помнить, что события "пересечение отрезков" появляются динамически, то есть мы должны уметь вставлять в очередь новые элементы, а также некоторые события могут быть вычислены больше одного раза, но в очереди при этом они должны встречаться лишь единожды. И ещё нам нужна операция поиска следующего события в очереди. Мы будем использовать сбалансированное бинарное дерево поиска, в котором обе операции - вставка нового элемента и поиск и удаление следующего элемента - занимают $O(\log{m})$ времени, где $m$ - количество событий в $\Omega$. 

Статус заметающей прямой нужен нам для поиска соседей данного отрезка. Структура данных статуса должна быть динамической, так как отрезки добавляются и удаляются из него в ходе работы алгоритма. Отрезки в статусе упорядочены, поэтому мы снова будем использовать сбалансированное бинарное дерево поиска. 

Итак, мы придумали эффективный алгоритм, сильно сократив количество проверок (строгая оценка будет даначуть позже). Теперь нужно убедиться в том, что он по-прежнему находит все точки пересечений. Для начала уберем из рассмотрения перекрывающиеся (имеющие больше одной общей точки) и вертикальные отрезки, а также случай, когда больше двух отрезков пересекаются в одной точке. Эти три случая обрабатываются несложно, не выбиваясь из общего алгоритма. Однако для простоты рассуждений временно будем считать, что таких ситуаций быть не может, и покажем, что в таком предположении алгоритм работает  корректно.

### Теорема
> Пусть $s_i$ и $s_j$ - два невертикальных пересекающихся в точке $p$ отрезка, причем $p$ - единственная их общая точка. Также предположим, что нет третьего отрезка, проходящего через $p$. Тогда существует событие, которое случится раньше, чем заметающая прямая $l$ достигнет $p$, при котором $s_i$ и $s_j$ станут соседями и будут проверены на пересечение.

$\triangleright$ <div style="padding-left:40px"> 
Пусть $l$ находится строго левее $p$. Если $l$ достаточно близка к $p$, то $s_i$ и $s_j$ должны быть соседями (более формально мы должны взять такое положение $l$, при котором между ней и вертикальной прямой, параллельной оси $Y$ и проходящей через точку $p$, нет событий). Другими словами, существует такое положение заметающей прямой, в котором отрезки будут соседями. С другой стороны $s_i$ и $s_j$ не были соседями, когда $l$ начинала свое движение, поскольку она стартует левее всех точек. Следовательно, по ходу ее движение должно было произойти событие $q$, при котором $s_i$ и $s_j$ стали соседями и были проверены на пересечение.
</div>
$\triangleleft$

Отметим, что началом отрезка будем считать тот его конец, который меньше второго в лексикографическом порядке.

Теперь вернемся к трем случаям, о которых мы временно забывали, и рассмотрим, как обрабатывается каждый из них:
1. <img src="images/skew.png" style="float: right"/> Если в множестве $S$ есть вертикальный отрезок $s_i$, то в очереди $\Omega$ у нас будут лежать два события с одинаковыми $x$-координатами, отвечающие за начало и конец $s_i$. Благодаря вышеопределенному порядку, сначала будет рассмотрено событие "начало отрезка", так как его $y$-координата меньше, и отрезок будет проверен на пересечение с соседями. Через какое-то время будет рассмотрено событие "конец отрезка" и будут проведены соответствующие проверки. То есть случай вертикального отрезка ничем не отличается от случая невертикального, и нам не нужно совершать никаких дополнительных действий для его обработки. Для лучшего понимания можно просто представить, что заметающая прямая не вертикальная, а наклонена от вертикальной оси под очень небольшим углом влево, как на рисунке справа (такой прием называется *skew*). Тогда очевидно, что нижний конец вертикального отрезка всегда будет достигаться первым. <br><br>
2. Для случая, когда два отрезка накладываются друг на друга, мы просто заранее должны решить, что мы хотим от алгоритма в таких ситуациях. То есть для корректной обработки в этом случае нужно четкое определение того, что ожидается на выходе. Это может быть любая точка из общих точек двух отрезков, может быть конкретно обозначенная. Кроме того может быть кому-то понадобится возвращать список пар пересекающихся отрезков, в таком случае это опять же не будет выбиваться из общей картины. <br><br>
3. <img src="images/several_intersections_case.png" style="float: right"/> Теперь разберем случай, когда в одной точке пересекается больше двух отрезков. Так как у нас события отсортированы в порядке "начало отрезка" < "пересечение отрезков" < "конец отрезка", то случаи, когда в точке пересечения отрезков начинаются или заканчиваются другие отрезки, вписывается в общую картину и не требуют отдельного рассмотрения. Теперь вспомним, что в точке пересечения мы меняем порядок пересекающихся отрезков в статусе и проверяем их на пересечение с новыми соседями. Поэтому сколько бы отрезков не пересекалось в точке, благодаря таким перестановкам все пары пересекающихся отрезков будут найдены. Рассмотрим пример справа. Кратко опишем действия алгоритма: сначала в пустой статус добавляется отрезок $s_1$, далее - отрезок $s_2$, после проверки находится пересечение $s_2$ с $s_1$, далее в статус добавляется $s_3$ и находится пересечение $s_3$ с $s_1$. После этого добавляются $s_4$, $s_5$ и $s_6$, при добавлении последнего находятся пересечения $s_6$ с $s_1$ и $s_6$ с $s_3$. Если бы мы не рассматривали события "пересечение отрезков", то далее алгоритм прошел бы по событиям "конец отрезка", не нашел бы больше пересечений (поскольку мы берем только пересечения правее) и завершил свою работу. Но так как у нас три типа событий, то после добавления $s_6$ алгоритм перейдет к обработке события "пересечение отрезков $s_2$ и $s_1$", и проверит на пересечение $s_2$ и $s_3$ (найдет новое пересечение) и $s_1$ и $s_4$, далее будут обработаны события "пересечение $s_1$ и $s_3$" и "пересечение $s_6$ и $s_1$". Во время проверки последнего $s_6$ и $s_1$ поменяются местами и на пересечение будут проверены пары $s_6$ с $s_2$ (найдет новое пересечение) и $s_1$ с $s_3$. Далее будут проверены события "пересечение $s_6$ и $s_3$" (указанные отрезки поменяются местами и будут проверены пары $s_3$ с $s_1$ и $s_6$ с $s_5$), "пересечение $s_6$ и $s_2$", после чего отрезки в порядке $s_2$, $s_4$, $s_3$, $s_5$, $s_1$, $s_6$ будут удалены из статуса. Таким образом, алгоритм найдет шесть пар пересекающихся отрезков.

### Реализация алгоритма
Ранее мы подробно обсудили идею алгоритма, теперь давайте соберем все вместе и начнем практическую реализацию. Здесь будет дан каркас основных моментов и описание их ожидаемой функциональности, читателю в качестве тренировки предлагается самому дописать недостающие части. Входные данные содержатся в файле *segments_set.in* в следующем формате: в первой строке задано $n$ - количество отрезков в множестве $S$, в каждой из следующих $n$ строк указаны координаты начала и конца соответствующего отрезка.

**Input:** Множество отрезков на плоскости $S$.<br>
**Output:** Множество точек пересечения отрезков из $S$.

Итак, опишем алгоритм *in general*, пока не рассматривая обработку событий и другие детали: 
1. Инициализируем $status$ пустым множеством.
2. Инициализируем множество событий $events$ пустым множеством.
3. Вставим в $events$ события начал и концов отрезков из $S$.
4. Пока очередь событий не пуста, будем делать следующее:
 1. Достаем из очереди следующее событие $p$
 2. Обрабатываем $p$
 
Для простоты реализации можно просто удалять пройденные события и на каждом шаге брать минимальное. Реализация этой части может быть следующей:

In [ ]:
Point = entities.Point
Segment = entities.Segment
Event = entities.Event
Type = entities.Type

def find_intersections(segments):
    status = bintrees.RBTree()
    events = bintrees.RBTree()
    for s in segments:
        events.insert(Event(s, Type.left), Event(s, Type.left))
        events.insert(Event(s, Type.right), Event(s, Type.right)) 
    while not events.is_empty():
        event, _ = events.pop_min()
        handle_event_point(event)

segments = []
with open("segments_set.in") as inp:
    n = int(inp.readline())  # number of segments
    for i in range(0, n):
        x1, y1, x2, y2 = map(float, inp.readline().split())
        s = Segment(Point(x1, y1), Point(x2, y2), i)
        segments.append(s)
        plt.plot([x1, x2], [y1, y2], c = 'black')
        plt.scatter(x1, y1, c = 'black', s = 30)
        plt.scatter(x2, y2, c = 'black', s = 30)
        
find_intersections(segments)
        
plt.show()       

Как видно из вышеприведенного кода, за обработку событий у нас будет отвечать функция *handle_event_point(event)*. В ней нужно будет обрабатывать каждый тип событий: проверять соседние отрезки на пересечение и искать новые события. Для реализации данной функции нам понадобится дополнительная, вычисляющая пересечение двух отрезков.

Из курса линейной алгебры мы знаем, что точку пересечения $q$ двух непараллельных отрезков $a_1 b_1$ и $a_2 b_2$ можно найти следующим образом:

\begin{equation} 
\left( \begin{array}{cc}
   a_1.y - b_1.y & b_1.x - a_1.x\\
   a_2.y - b_2.y & b_2.x - a_2.x
\end{array} \right) 
\cdot 
\left( \begin{array}{c}
   x\\
   y
\end{array} \right) =
\left( \begin{array}{c}
   (a_1.y - b_1.y) \cdot a_1.x + (b_1.x - a_1.x) \cdot a_1.y\\
   (a_2.y - b_2.y) \cdot a_2.x + (b_2.x - a_2.x) \cdot a_2.y
\end{array} \right) \end{equation}

\begin{equation} \left( \begin{array}{c}
   x\\
   y
\end{array} \right) =  
\frac{
\left( \begin{array}{cc}
b_2.x - a_2.x & a_1.x - b_1.x\\
b_2.y - a_2.y & a_1.y - b_1.y
\end{array} \right) 
\cdot
\left( \begin{array}{c}
b_1.x \cdot a_1.y - b_1.y \cdot a_1.x\\
b_2.x \cdot a_2.y - b_2.y \cdot a_2.x
\end{array} \right) 
}{
\left| \begin{array}{cc}
a_1.y - b_1.y & b_1.x - a_1.x\\
a_2.y - b_2.y & b_2.x - a_2.x
\end{array} \right| } \end{equation}

Предположим, что два отрезка пересекаются (это, как мы знаем из конспекта про афинное пространство, легко проверяется с помощью предиката поворота и *bounding-box'а*). Сейчас Вам предлагается написать функцию, возвращающую точку пересечения этих отрезков. Для начала давайте будем работать в целых числах, то есть координаты концов отрезков будут целыми числами в $R^2$, а функция должна будет вернуть точку пересечения в однородных координатах. Если отрезки имеют больше одной точки пересечения, то вывести любую.

In [ ]:
def get_intersection_point(a, b, c, d):   # концы отрезков в формате np.array([x, y])
    # Заполните тело функции. Она должна возвращать точку пересечения
    # отрезков ab и cd в однородных координатах.
    return solutions.get_intersection_point(a, b, c, d)  # точка пересечения в формате np.array([x, y, det])

Чтобы убедиться в корректности написанной Вами функции, запустите ее на наборе тестов:

In [ ]:
checker.check_first_exercise(plt, get_intersection_point)

// TODO
* упр на локализационную структуру или сортировку событий фильтрованным вычислением поворота (?)
* handle_event_point
* find_new_event

In [ ]:
def handle_event_point(event):
    return 0;

### Теорема (корректность алгоритма)
>Алгоритм находит все точки пересечения и отрезки, которые их содержат. 

$\triangleright$ <div style="padding-left:40px"> 
Воспользуемся индукцией по событиям, отсортированным в вышеуказанном порядке. Предположим что все события $q : q < p$ были обработаны корректно. Докажем, что $p$ также будет корректно обработано и все отрезки, содержащие $p$, будут найдены. 
1. Если событие $p$ - точка пересечения отрезков, то по предыдущей теореме найдется предшествующее $p$ событие $q$ такое, что при его обработке отрезки, пересекающиеся в $p$, будут соседними в статусе. Следовательно, в этом случае $p$ будет обнаружена.
2. Если $p$ - начало или конец отрезка, то она была добавлена в $events$ в самом начале работы алгоритма. Все содержащие её отрезки из статуса в момент обработки $p$ будут найдены. Для остальных отрезков, содержащих $p$, верно, что $p$ — их левый конец, и они будут найдены, так как мы храним их вместе с $p$ в $events$.
</div>
$\triangleleft$

Следовательно, после того, как заметающая прямая пройдет всю плоскость, а точнее после того, как будет рассмотрено последнее событие, мы вычислим все точки пересечений отрезков из множества $S$. 

### Теорема (оценка времени работы)
>Время работы алгоритма - $O((n + I)\log{n})$, где $I$ - количество пересечений.

$\triangleright$ <div style="padding-left:40px"> 
На первом шаге алгорим строит множество событий $events$, при использовании сбалансированного бинарного дерева в качестве структуры данных это выполняется за $O(n\log{n})$. Инициализация статуса занимает константное время. 

Далее на каждой итерации мы обрабатываем по одному событию. Обработка может включать в себя удаление отрезка из статуса за $O(\log{n})$ и проверку на пересечение отрезка с не более, чем двумя соседями, что может вызвать вставку новых событий (снова $O(\log{n})$). Всего при обработке события $p$ мы $m(p) = \left| L(p) \cup C(p) \cup U(p) \right|$ раз выполняем операции вставки, удаления и поиска отрезка в статусе, где $L(p)$ - множество отрезков, верхний конец которых есть $p$, $C(p)$ - множество отрезков, внутри которых содержится $p$, и $U(p)$ - множество отрезков, нижний конец которых есть $p$. Пусть $m = \sum_{p}^{}m(p)$. Тогда время работы алгоритма - $O(n\log{n} + m\log{n})$. Осталось показать, что $m = n + I$. 

Рассмотрим планарный граф, вершинами которого являются концы отрезков, а также их точки пересечения, а ребрами - части отрезков, их соединяющие. Рассмотрим событие $p$. Ему соответствует вершина графа со степенью $O(m(p))$. Следовательно, $m$ не превосходит суммы степеней всех вершин графа: $m = O(\sum_{p}^{}deg(p))$. Каждое ребро графа добавляет $+1$ к степени ровно двух вершин, из чего следует, что $m = O(2\left| E \right|) = O(\left| E \right|)$, где $\left| E \right|$ - количество ребер в графе. Пусть $\left| V \right|$ - количество вершин в графе, тогда $\left| V \right| \leqslant 2n + I$. Мы знаем, что в планарном графе $\left| E \right| = O(\left| V \right|)$. Таким образом, $O(\left| E \right|) = O(\left| V \right|) = O(2n + I)$. Следовательно, время работы алгоритма - $O((n + I)\log{n})$.
</div>
$\triangleleft$

### Требуемая память
Благодаря данному алгоритму можно сократить количество проверяемых отрезков с $n^2$ до $n + I$, что является отличным результатом. Теперь оценим затраты по памяти. Очевидно, что статус не может занимать больше $O(n)$ памяти, а вот очередь событий вполне может занимать $O(n + I)$, то есть если все пересечения окажутся правее заметающей прямой, то для очереди потребуется $O(n^2)$ памяти. Чтобы избежать этой оценки, поступают следующим образом: если два отрезка перестали быть соседями, то событие, отвечающее за их точку пересечения, удаляется из статуса. Таким образом, мы будем хранить не больше $O(n)$ событий, а на время работы алгоритма такая оптимизация не повлияет.

### Заключение
Итак, мы научились решать задачу о нахождении всех точек пересечения множества отрезков за $O((n + I)\log{n})$ времени и $O(n)$ памяти. Понятно, что в худшем случае данный алгоритм работает за $O(n^2\log{n})$. Существуют алгоритмы, которые имеют лучшую временную оценку, например, в следующей статье описывается решение поставленной задачи за время $O(n\log{n} + I)$ с сохранением оценки по памяти $O(n)$:

* [Ivan J. Balaban - An optimal algorithm for finding segments intersections](https://pdfs.semanticscholar.org/f523/84b8cc2b44f91c1e4048c7bc0021f6e01b01.pdf)

Однако алгоритмы Бентли-Оттмана и Ивана Балабана сложны в реализации, поэтому на практике используются достаточно редко. Чаще всего эта задача решается с использованием [квадродеревьев](https://github.com/CT-18/cg/tree/1_skip_quadtree), так как такой подход имеет ту же асимптотику и при этом более прост в реализации.